# SKY130 MiM Capacitor - Full Flow

Demonstrates the complete analog design flow using `CapRunner`:
1. Pre-layout simulation
2. Layout generation and GDS export  
3. DRC / LVS / PEX
4. Post-layout simulation (PEX switched on via kwargs)
5. Comparison

In [1]:
import sys
sys.path.insert(0, '..')
from pdk.sky130.config import config  # Module-level singleton

## 1. Pre-Layout Simulation

In [2]:
from src.runners.cap_runner import CapRunner

runner = CapRunner(w=10, l=10)
result_pre = runner.run_and_evaluate('prelayout')
print(f"Pre-layout capacitance at 1MHz: {result_pre['C_1MHz']*1e15:.1f} fF")

[INFO] Netlist written to /Users/fredrief/projects/pade/examples/tutorials/../sim_data/prelayout/CapacitorAC.spice
[INFO] Running: ngspice -b /Users/fredrief/projects/pade/examples/tutorials/../sim_data/prelayout/CapacitorAC.spice -r /Users/fredrief/projects/pade/examples/tutorials/../sim_data/prelayout/output.raw


[INFO] NGspice complete in 12.6s


Pre-layout capacitance at 1MHz: 206.0 fF


## 2. Layout & Verification

In [3]:
from pade.layout import UM
from pade.backends.gds.layout_writer import GDSWriter
from pdk.sky130.primitives.capacitors.layout import CapMimM4Layout
from pdk.sky130.layers import sky130_layers

cap_layout = CapMimM4Layout('C1', width=10*UM, height=10*UM)

gds_writer = GDSWriter(layer_map=sky130_layers)
gds_path = gds_writer.write(cap_layout, config.layout_dir)
print(f"Layout: {cap_layout.cell_name}, ports: {list(cap_layout.ports.keys())}")
print(f"GDS: {gds_path}")

Layout: cap_mim_m4, ports: ['MINUS', 'PLUS']
GDS: /Users/fredrief/projects/pade/examples/tutorials/../layout/cap_mim_m4.gds


### DRC / LVS / PEX

In [4]:
from utils.design_runner import DesignRunner
from pdk.sky130.primitives.capacitors.schematic import CapMimM4

cap_schematic = CapMimM4('cap_mim_m4', None, w=10, l=10)
design = DesignRunner(cap_layout, cap_schematic)

# Run all verification
result = design.run_all()
print(result)

DRC PASS: 0 errors

                         #       ###################       _   _
                        #        #                 #       *   *
                   #   #         #     CORRECT     #         |
                    # #          #                 #       \___/
                     #           ###################

PEX extraction successful
Netlist: /Users/fredrief/projects/pade/examples/tutorials/../pex/cap_mim_m4/pex_rc.spice


## 3. Post-Layout Simulation (with PEX)

In [5]:
# Same runner, just add pex kwarg - no code changes needed!
result_post = runner.run_and_evaluate('postlayout', pex={'C1': 'rc'})
print(f"Post-layout capacitance at 1MHz: {result_post['C_1MHz']*1e15:.1f} fF")

[INFO] PEX: C1 -> pex_rc.spice
[INFO] Netlist written to /Users/fredrief/projects/pade/examples/tutorials/../sim_data/postlayout/CapacitorAC.spice
[INFO] Running: ngspice -b /Users/fredrief/projects/pade/examples/tutorials/../sim_data/postlayout/CapacitorAC.spice -r /Users/fredrief/projects/pade/examples/tutorials/../sim_data/postlayout/output.raw


[INFO] NGspice complete in 12.8s


Post-layout capacitance at 1MHz: 214.8 fF
